In [1]:
'''Basic package'''
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm #進度條
import time
np.set_printoptions(formatter={'float_kind':'{:f}'.format})



In [2]:
#準備資料夾路徑
month=['03','04','05']
#load all data
full_data = pd.DataFrame()
#loop 3個資料夾
for i in np.arange(len(month)):
    dir_list=[]
    #抓裡面的資料清單，因為其中會有一個'.ipynb_checkpoints'檔，所以只選csv結尾的
    for file in os.listdir("file_stat_%s/" %month[i]):
        if file.endswith(".csv"):
            dir_list.append(file)
    #只是檢查用可以省略
    dir_list.sort()
    #抓取每個清單裡的檔案，加到同一個df下面
    for j in tqdm(np.arange(len(dir_list)), desc="full_data"):
        new_data = pd.read_csv(os.path.join("file_stat_%s/" %month[i],dir_list[j]))
        new_data = new_data.drop(new_data.columns[0], axis=1)
        full_data=pd.concat([full_data,new_data])
    print('folder_%s done' % month[i])

HBox(children=(IntProgress(value=0, description='full_data', max=31), HTML(value='')))


folder_03 done


HBox(children=(IntProgress(value=0, description='full_data', max=30), HTML(value='')))


folder_04 done


HBox(children=(IntProgress(value=0, description='full_data', max=31), HTML(value='')))


folder_05 done


In [3]:
#因為要做sum加總，所以先將所有頻率移除，晚點再用計算的補回來，增加day_count的feature
full_data['day_count']=[1]*full_data.shape[0]
full_data=full_data.groupby(by='FileID').sum()
#group玩FileID會跑到Index，把他補到後面的column
full_data['FileID']=full_data.index

In [4]:
#用法可參考：https://stackoverflow.com/questions/14529838/apply-multiple-functions-to-multiple-groupby-columns
file_freq = full_data.iloc[:,0:2]
f = {'file_freq':['min','max','mean','std']}
merge=file_freq.groupby('FileID').agg(f)
merge['FileID']=merge.index
merge.index=[0] * merge.shape[0]
merge.columns=['file_freq_min', 'file_freq_max', 'file_freq_mean','file_freq_std','FileID']
merge=merge[['FileID','file_freq_min', 'file_freq_max', 'file_freq_mean','file_freq_std']]
#補缺失值（來源因如果file_freq=1，沒有std需設為0
merge.file_freq_std=merge.file_freq_std.fillna(0)
#return false>>補值完成
merge.isnull().values.any()

False

In [5]:
#兩組df合在一起
merge = pd.merge(merge,full_data,how='outer',on=['FileID'])
#把user_freq跟prod_freq補回
merge['user_freq']=merge['file_freq']/merge['user_count']
merge['prod_freq']=merge['file_freq']/merge['prod_count']


In [6]:
merge.to_csv('total_data_analysis.csv',index=False )